In [2]:
###Importing the right packaages / aliasing / room folder look ups

import pandas as pd
from workspaces.workspaces import *
import datetime as dt
import os
import csv
import io
import zipfile

# Variables
fp = 'C:/Users/JM/Documents/hi-data-upload-utility-1.6/bin/Test_files/Processed/'
date = dt.date.today()
file_release = dt.datetime.strftime(date,'%Y-%m-%d')
w = Workspaces()

afile = [] 
bifile = []
bpfile = [] 

def claim_combine(file):
    if ('~ST*835' in file) and ('ISA*' in file):
        afile.append(file)
    elif ('~ST*837' in file) and ('ISA*' in file) and ('005010X222A1' not in file):
        bifile.append(file)
    elif ('~ST*837' in file) and ('ISA*' in file) and ('005010X222A1' in file):
        bpfile.append(file)
    else:
        print('not a claim')

def claim_clean(file):
    file = [w.replace('!','*') for w in file]
    file = [w.replace('\\|',':') for w in file]
    file = [w.replace('>',':') for w in file]
    file = [w.replace('[\r\n]',' ') for w in file]
    return file




In [3]:
### test file discovery 
## set this cell to whatever folder you are checking for newfiles
get_rooms = w.get_rooms()
room_info = w.get_room_info(350519)
test_room_room_folders = w.get_room_folders(350519)
##test_room_room_folders['subFolders'][1]['subFolders']

#test_837 newfiles folderid = 5482329, masterfolder = 5482328, processed =5482327
doc = w.get_documents(room_id = 350519,
                        folder_id = 5482329) ##test 837s new files folder
doc = pd.DataFrame(doc)

if len(doc) == 0:
    print('folder is empty on '+ str(file_release))
    exit()
else:
    print(doc)

                                   guid                             file_name  \
0  7a7ff19c-97e4-4324-8306-8ff9d136136c                                test_3   
1  6bfb4a6f-28a9-43bb-a10c-e2a6c7159801  86662349-7a0b-47ae-a18a-564f8a43a15c   
2  423226da-afc5-443e-bf54-60a7d046738b  b6802573-eaa1-4ed3-b286-78d1b5d5b751   
3  8b033353-4162-4efa-88d2-28641eddbb70  aaf9adae-0816-4b96-b171-435f1d08e2f9   
4  2eb8f25e-5f94-424a-8795-8ca0c238edd5                      Unadj File 5.zip   

                     sender        date_of_submission  \
0  ethelen@millenniumcc.org  2020-05-15T11:31:57-0400   
1  ethelen@millenniumcc.org  2020-05-15T11:50:52-0400   
2  ethelen@millenniumcc.org  2020-05-15T11:50:54-0400   
3  ethelen@millenniumcc.org  2020-05-15T11:50:56-0400   
4  pgagnon@millenniumcc.org  2020-05-26T13:42:56-0400   

          modification_date file_type  
0  2020-05-15T11:31:57-0400            
1  2020-05-15T11:50:52-0400            
2  2020-05-15T11:50:54-0400            
3  2020-05

In [4]:
##creating a new folder in processed(file release folder)
os.mkdir(fp+str(file_release))
w.create_folder(folder_id=5482327,
                room_id=350519,
                new_folder_name=str(file_release))


<Response [200]>

In [5]:
## figuring out the file path for that new folder so we can send the processed files correct release folder
test_room_room_folders = w.get_room_folders(350519)
for i in test_room_room_folders['subFolders']:
    if i['name'] == '837s':
        temp = i['subFolders']
        for x in temp:
            if x['name'] == 'Processed':
                temp_2 = x['subFolders']
                for y in temp_2:
                    if y['name'] == str(file_release):
                        processed_file_release_path = y['fullPath']
                        processed_file_release_id = y['id']
                
print(processed_file_release_path)
print(processed_file_release_id)

###test_room_room_folders['subFolders'][1]['subFolders'][2]

837s/Processed/2020-06-05
5604807


In [6]:
#gathering files in the foler // combining all claims into one and moving original files into processed folder

for index, row in doc.iterrows():
    if row['file_type'] == 'zip':
        zip_document = w.download_document(row['guid'])
        zip_bytes = io.BytesIO(zip_document)
        myzipfile = zipfile.ZipFile(zip_bytes)
        for name in myzipfile.namelist():
            foofile = myzipfile.open(name).read().decode("ascii")
            claim_combine(foofile)
            desktop_process = open(fp+ str(file_release)+ '/'+ name+ '.txt', 'w')
            desktop_process.write(foofile)
            desktop_process.close()
        print('zip file')
        ##w.move_document(room_id=350519, document_id=row['guid'], folder_path='837s/Processed')
        w.move_document(room_id=350519, document_id=row['guid'], folder_path=processed_file_release_path)
    else:
        reg_file = w.download_document(row['guid']).decode("ascii")
        desktop_process_1 = open(fp+ str(file_release)+ '/'+ row['file_name']+ '.txt', 'w')
        desktop_process_1.write(reg_file)
        desktop_process_1.close()
        claim_combine(reg_file)
        print('reg file')
        ##w.move_document(room_id=350519, document_id=row['guid'], folder_path='837s/Processed')
        w.move_document(room_id=350519, document_id=row['guid'], folder_path=processed_file_release_path)
        
    



reg file
reg file
reg file
reg file
zip file


In [7]:
##Taking the combined file lists and creating one single file and moving it to master bin folder
## create file path for strings and add in room numbers to settings file 

afile = claim_clean(afile)
bifile = claim_clean(bifile)
bpfile= claim_clean(bpfile)

if len(bifile) >= 1:
    with open('C:\\Users\\JM\\Documents\\hi-data-upload-utility-1.6\\bin\\Test_files\\'+'Master_bi'+file_release, 'w' ) as f:
        for i in bifile:
            f.write(str(i))
else:
    print('Bi file is empty')
      
if len(bpfile) >= 1:
    with open('C:\\Users\\JM\\Documents\\hi-data-upload-utility-1.6\\bin\\Test_files\\'+'Master_bp'+file_release, 'w' ) as f:
        for i in bpfile:
            f.write(str(i))
            
else:
    print('Bp file is empty')
    
if len(afile) >= 1:
    with open('C:\\Users\\JM\\Documents\\hi-data-upload-utility-1.6\\bin\\Test_files\\'+'Master_a'+file_release, 'w' ) as f:
        for i in bifile:
            f.write(str(i))
            
else:
    print('Afile is empty')
    
w.session_logout()

Bp file is empty
Afile is empty


<Response [401]>

In [ ]:
import hiduu.hiduu as h
import datetime as dt

# Variables
##fp = '/Documents/hi-data-upload-utility-1.6/bin/Files/OUA'
date = dt.date.today()
file_release = dt.datetime.strftime(date,'%Y%m%d')

# h.hiduu_upload_win(hiduu_path = HIDUU_PATH,
#                      upload_command = 'uploadDataSetFile',
#                      system_account_id = vertica_sys_acct_id,
#                      system_account_secret = vetted_system_secret,
#                      source_id = vertica_sync_id,
#                      data_set_id = FILE_NAME6,
#                      file_id = file_type_id,
#                      file_release = file_release,
#                      spec_version = sv,
#                      file = None)

h.hiduu_upload_win()